In [7]:
import xlwings as xw
import pandas as pd
from ezdxf.select import Polygon
from shapely.geometry.linestring import LineString
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.multilinestring import MultiLineString
from shapely.wkt import loads
import ast
from shapely.geometry import Point
from win32com.universal import com_error

In [ ]:
sheet_result = 'Зона 2'

In [ ]:
sheet_autocad = xw.sheets['Ведомость ОРМ']
taxation_list_orm_df = sheet_autocad.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
taxation_list_orm_df['Позиция номера'] = taxation_list_orm_df['Позиция номера'].apply(lambda x: loads(x))
taxation_list_orm_df['Геометрия'] = taxation_list_orm_df['Геометрия'].apply(lambda x: loads(x))

taxation_list_orm_df

In [ ]:
sheet_zones = xw.sheets['Зоны']
zones_df = sheet_zones.range('A1').expand().options(pd.DataFrame, header=1, index=False).value

zones_df

In [ ]:
zone_names = zones_df['Наименование'].tolist()
zone_names.remove(sheet_result)

zone_names

In [ ]:
used_split_numbers_df = pd.DataFrame(columns=['Исх.номер', 'Список геометрии'])

for zone_name in zone_names:
    try:
        sheet_zone = xw.sheets[zone_name]
    except com_error:
        continue
    if sheet_zone.range('A1').value is None:
        continue
    _used_split_numbers_df = sheet_zone.range('A1').expand().options(pd.DataFrame, header=1, index=False).value[
        ['Исх.номер', 'Геометрия']]
    _used_split_numbers_df['Геометрия'] = _used_split_numbers_df['Геометрия'].apply(lambda x: loads(x))
    used_split_numbers_df = pd.concat([used_split_numbers_df, _used_split_numbers_df])

if len(used_split_numbers_df) != 0:
    used_split_numbers_df['type'] = used_split_numbers_df['Геометрия'].apply(lambda x: isinstance(x, Point))
    used_split_numbers = used_split_numbers_df[used_split_numbers_df['type']]['Исх.номер'].tolist()
else:
    used_split_numbers = []

used_split_numbers

In [ ]:
taxation_list_orm_df_not_used = taxation_list_orm_df[~taxation_list_orm_df['Номер точки'].isin(used_split_numbers)]

taxation_list_orm_df_not_used

In [ ]:
zone_shape = loads(zones_df[zones_df['Наименование'] == sheet_result]['Геометрия'].tolist()[0])

zone_shape

In [ ]:
def get_size_text(geometry: Point | Polygon | MultiPolygon | LineString | MultiLineString, quantity: str) -> str:
    if isinstance(geometry, Polygon) or isinstance(geometry, MultiPolygon):
        return f"{round(geometry.area, 1)} м.кв."
    elif isinstance(geometry, LineString) or isinstance(geometry, MultiLineString):
        return f"{round(geometry.length, 1)} м.п."
    else:
        try:
            assert int(quantity)%1 == 0
            return str(int(float(quantity)))
        except:
            return quantity

In [ ]:
intersections_shapes = []
for _, series in taxation_list_orm_df_not_used.iterrows():
    geometry = series['Геометрия']
    intersection = zone_shape.intersection(geometry)
    if intersection:
        if isinstance(intersection, Point) or isinstance(intersection, Polygon) or isinstance(intersection, LineString):
            intersections_shapes.append({
                'Исх.номер': series['Номер точки'],
                'Номер': None,
                'Наименование': series['Наименование'],
                'Количество': get_size_text(intersection, series['Количество']),
                'Высота': series['Высота'],
                'Толщина': series['Толщина'],
                'Состояние': series['Состояние'],
                'Кустарник': series['Кустарник'],
                'Позиция номера': intersection.centroid,
                'Геометрия': intersection
            })
        elif isinstance(intersection, MultiPolygon) or isinstance(intersection, MultiLineString):
            number_suffix = 0
            for intersection_part in intersection.geoms:
                number_suffix += 1
                intersections_shapes.append({
                    'Исх.номер': series['Номер точки']+'_'+str(number_suffix),
                    'Номер': None,
                    'Наименование': series['Наименование'],
                    'Количество': get_size_text(intersection_part, series['Количество']),
                    'Высота': series['Высота'],
                    'Толщина': series['Толщина'],
                    'Состояние': series['Состояние'],
                    'Кустарник': series['Кустарник'],
                    'Позиция номера': intersection_part.centroid,
                    'Геометрия': intersection_part
                })

intersections_shapes_df = pd.DataFrame(intersections_shapes)

intersections_shapes_df

In [ ]:
sheet = xw.sheets[sheet_result]
for l in ['A', 'B']:
    sheet[f'{l}:{l}'].number_format = '@'
sheet.range('A1').value = intersections_shapes_df
sheet["A1"].value = ['index']

In [17]:

selected_cells = xw.apps.active.selection
print("get selections")
active_sheet = xw.sheets.active
rows = active_sheet.api.Rows
print("get rows")

get selections
get rows


In [16]:
pd.Series(rows)

com_error: (-2147352570, 'Неизвестное имя.', None, None)

In [15]:
hidden_rows = {row.Index for row in rows if row.Hidden}
print("get hidden rows")

KeyboardInterrupt: 

In [ ]:
not_hidden_cells = [
            cell for cell in selected_cells if cell.row not in hidden_rows
        ]
not_hidden_cells